<a href="https://www.kaggle.com/code/smellytofu/delfttreescnn?scriptVersionId=168847286" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#to clear outputs

import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
import scipy


img = image.load_img("/kaggle/input/tree-type-classification-main/tree-type-classification-main/Model/is_tree_images/train_images/tree/152_2021-09_52-009821571910685_4-325297593030579_Redelijk.jpg")
plt.imshow(img)
cv2.imread("/kaggle/input/tree-type-classification-main/tree-type-classification-main/Model/is_tree_images/train_images/tree/152_2021-09_52-009821571910685_4-325297593030579_Redelijk.jpg").shape

train = ImageDataGenerator(rescale= 1/255)
validation = ImageDataGenerator(rescale= 1/255)

train_dataset = train.flow_from_directory("/kaggle/input/tree-type-classification-main/tree-type-classification-main/Model/is_tree_images/train_images", target_size = (200,200), batch_size = 3, class_mode = 'binary')

validate_dataset = validation.flow_from_directory("/kaggle/input/tree-type-classification-main/tree-type-classification-main/Model/is_tree_images/validate_images",target_size = (200,200), batch_size = 3, class_mode = 'binary')
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics = ['accuracy'])
model_fit = model.fit(train_dataset,
                      steps_per_epoch = 20,
                      epochs = 30,
                      validation_data = validate_dataset)

img = image.load_img("/kaggle/input/tree-type-classification-main/tree-type-classification-main/Model/is_tree_images/validate_images/tree/13182_2021-09_52-01082266294491_4-355730754059023_Slecht.jpg", target_size=(200,200))
plt.imshow(img)
plt.show()

X = image.img_to_array(img)
X = np.expand_dims(X,axis = 0)
images = np.vstack([X])
val = model.predict(images)
if val == 0:
    print("not a tree")
else:
    print("a tree")
tf.keras.models.save_model(model,"/kaggle/working/is_tree_model.keras", 'is_tree_model')

In [ ]:
#classify tree/notree

from pathlib import Path 
import tensorflow as tf
from tensorflow.keras.preprocessing import image  # Importing from tf.keras
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
loaded_model = tf.keras.models.load_model("/kaggle/working/is_tree_model.keras")
TREE_TYPES = ['Dood', 'Goed', 'Matig', 'Redelijk', 'Slecht', 'Zeer Slecht']

Path('/kaggle/working/classified_images/').mkdir(parents=True, exist_ok=True)

FOLDER_PATH_OUTPUT = '/kaggle/working/classified_images/'
FOLDER_PATH_INPUT = '/kaggle/input/trees-by-category/images/'

for tree_path in TREE_TYPES:   
# Construct the full path for the current tree type
    output_tree = os.path.join(FOLDER_PATH_OUTPUT, 'tree', tree_path)
    output_not_tree = os.path.join(FOLDER_PATH_OUTPUT, 'not_tree', tree_path)



    # Check if the directory exists, if not, create it
    if not os.path.exists(output_tree):
        os.makedirs(output_tree)


    if not os.path.exists(output_not_tree):
        os.makedirs(output_not_tree)
    

for tree_path in TREE_TYPES:
    # Construct the full path for the current tree type
    output_tree = os.path.join(FOLDER_PATH_OUTPUT, 'tree', tree_path)
    output_not_tree = os.path.join(FOLDER_PATH_OUTPUT, 'not_tree', tree_path)
    input_tree_type = os.path.join(FOLDER_PATH_INPUT, tree_path)

    for filename in os.listdir(input_tree_type):
        image_input_path = os.path.join(input_tree_type, filename)
        if os.path.isfile(image_input_path):
            # Load image using TensorFlow's Keras preprocessing
            img = image.load_img(image_input_path, target_size=(200, 200))

            # Convert image to array
            X = image.img_to_array(img)
            X = np.expand_dims(X, axis=0)

            # Predict
            prediction = loaded_model.predict(X)

            if prediction == 0:
                shutil.copy(image_input_path, os.path.join(output_not_tree, filename))
            else:
                shutil.copy(image_input_path, os.path.join(output_tree, filename))
        

In [ ]:
#blurring images 
!pip install inference_sdk
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
from PIL import Image, ImageDraw, ImageFilter
import os

custom_configuration = InferenceConfiguration(confidence_threshold=0.1)

CLIENT = InferenceHTTPClient(
    api_url="https://outline.roboflow.com",
    api_key="kiW8upICXaWRP7kAm4Ct"
)

# Define the root folder where your images are stored
root_folder = "/kaggle/working/classified_images/tree"

# Iterate over each subfolder
for tree_cat in os.listdir(root_folder):
    tree_cat_path = os.path.join(root_folder, tree_cat)


    # Iterate over each image in the "tree" folder
    for img_file in os.listdir(tree_cat_path):
            img_path = os.path.join(tree_cat_path, img_file)

            with CLIENT.use_configuration(custom_configuration):
                results = CLIENT.infer(img_path, model_id="cameraexplorer/1")

            # Load the original image
            image = Image.open(img_path)

            # Create a blurred version of the original image
            blurred_image = image.filter(ImageFilter.GaussianBlur(15))  # Adjust the blur radius as needed

            # Initialize a blank mask with the same dimensions as the original image
            mask = Image.new('L', image.size, 0)
            draw = ImageDraw.Draw(mask)

            has_tree = False  # Initialize flag

            # Iterate over each detected object in your results
            for obj in results['predictions']:
                if obj['class'] == 'tree':
                    has_tree = True  # Initialize flag
                    # 'points' is a list of dictionaries, each with 'x' and 'y' keys
                    # Convert this into a list of tuples that PIL can use to draw the polygon
                    polygon = [(point['x'], point['y']) for point in obj['points']]

                    # Draw the polygon on the mask and fill it with white (255)
                    draw.polygon(polygon, fill=255)

            # Combine the original image with the blurred image, using the mask to keep the areas within the polygons clear
            final_image = Image.composite(image, blurred_image, mask)

            # Save
            if has_tree:
                final_image.save(img_path)
                print(f'Saving {img_path} in {tree_cat}')

In [ ]:
#transform and generate images for low sample datasets (after importing blurred_trees)
import os
from collections import Counter
from PIL import Image
import torchvision.transforms as transforms
import random

# Function to identify categories with lower samples
def identify_low_sample_categories(images_folder):
    image_counts = {}
    for cat_folder in os.listdir(images_folder):
        cat_path = os.path.join(images_folder, cat_folder)
        image_counts[cat_folder] = len(os.listdir(cat_path))
    max_samples = max(image_counts.values())
    low_sample_categories = [label for label, count in image_counts.items() if count < max_samples]
    return low_sample_categories

# Define data augmentation transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2))  # Random translation
])

# Function to apply transformations and save additional images
def augment_and_save_images(images_folder, low_sample_categories, transforms):

    image_counts = {}
    for cat_folder in os.listdir(images_folder):
        cat_path = os.path.join(images_folder, cat_folder)
        image_counts[cat_folder] = len(os.listdir(cat_path))
        
    cat_counts = Counter(image_counts)
    max_samples = max(cat_counts.values())
    for category in low_sample_categories:
        os.makedirs('/kaggle/working/'+"tree"+"/"+category)
        cat_folder = os.path.join(images_folder, category)
        num_samples = cat_counts[category]

        # Calculate how many additional images to generate
        num_additional_images = max_samples - num_samples
        for i in range (num_additional_images):
            random_index = random.randint(0, len(os.listdir(cat_folder)) - 1)
            randomfilename=os.listdir(cat_folder)[random_index]
            image_path = os.path.join(cat_folder, randomfilename)
            image = Image.open(image_path)

            # Apply transformations and save additional images
            transformed_image = transforms(image)
            new_filename = f"augmented_{num_samples + i + 1}_{randomfilename}"  # Append prefix to filename
            new_image_path = os.path.join("/kaggle/working/"+"tree/"+category, new_filename)
            transformed_image.save(new_image_path)
# Transform and generate new blurred images for categories with smaller samples
images_folder = '/kaggle/input/blurred-trees/classified_images/tree'
low_sample_categories = identify_low_sample_categories(images_folder)
augment_and_save_images(images_folder, low_sample_categories, augmentation_transforms)

In [ ]:
import os
import random
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from transformers import ViTForImageClassification
from scipy import stats
from PIL import Image, ImageDraw, ImageFilter

label_to_int = {'Dood': 0, 'Goed': 1, 'Matig': 2, 'Redelijk': 3, 'Slecht': 4,'Zeer Slecht': 5 }

class CustomImageDataset1(Dataset):
    def __init__(self, images, labels, transforms=None, samples_per_class=None):
        self.images = images
        self.labels = [label_to_int[label] for label in labels]
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]

        # Remove top 30% of the image
        width, height = image.size
        image = image.crop((0, height * 0.3, width, height))

        if self.transforms:
            image = self.transforms(image)

        return image, label

class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transforms=None, sampling_factors=None):
        self.images = images
        self.labels = labels
        self.transforms = transforms
        self.sampling_factors = sampling_factors if sampling_factors is not None else {label: 1 for label in set(labels)}

    def __len__(self):
        # Calculate total length considering sampling factors
        total_length = 0
        for label in self.labels:
            total_length += self.sampling_factors[label]
        return total_length

    def __getitem__(self, idx):
        # Find the actual image index based on the sampling factor
        actual_idx = idx
        for i, label in enumerate(self.labels):
            if actual_idx < self.sampling_factors[label]:
                break
            actual_idx -= self.sampling_factors[label]


        image = Image.open(self.images[i])
        label = self.labels[i]

        # Remove top 30% of the image
        width, height = image.size
        image = image.crop((0, height * 0.3, width, height))

        # Process image
        if self.transforms:
            image = self.transforms(image)

        return image, label_to_int[label]

def load_images(folder_path):
    print("Loading images...")
    images = []
    labels = []
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            images.append(img_path)
            labels.append(class_folder)
    print("Total images loaded:", len(images))
    return images, labels

def split_dataset(images, labels, train_ratio=0.6, val_ratio=0.2):
    # Split dataset into train, validation, and test
    print("Splitting dataset...")
    dataset = list(zip(images, labels))
    random.shuffle(dataset)
    train_size = int(len(dataset) * train_ratio)
    val_size = int(len(dataset) * val_ratio)
    train_set = dataset[:train_size]
    val_set = dataset[train_size:train_size + val_size]
    test_set = dataset[train_size + val_size:]
    print(f"Dataset split into {len(train_set)} training, {len(val_set)} validation, and {len(test_set)} test images.")
    return train_set, val_set, test_set

def count_class_distribution(dataset):
    class_counts = {}
    for _, label in dataset:
        class_counts[label] = class_counts.get(label, 0) + 1
    return class_counts

def calculate_sampling_factors(train_set):
    label_counts = Counter(label for _, label in train_set)
    min_samples = min(label_counts.values())
    print(min_samples)

    # Calculate sampling factor for each class
    sampling_factors = {label: round(min_samples / count) for label, count in label_counts.items()}
    return sampling_factors

def create_dataloaders(train_set, val_set, test_set, sampling_factors, batch_size):
    # Define transformations
    print("Creating dataloaders...")

    train_transforms = transforms.Compose([
        transforms.RandomCrop(224),  # Specify size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_test_transforms = transforms.Compose([
        transforms.RandomCrop(224),  # Specify size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    train_dataset = CustomImageDataset([i[0] for i in train_set], [i[1] for i in train_set], transforms=train_transforms, sampling_factors=sampling_factors)
    val_dataset = CustomImageDataset([i[0] for i in val_set], [i[1] for i in val_set], transforms=val_test_transforms)
    test_dataset = CustomImageDataset([i[0] for i in test_set], [i[1] for i in test_set], transforms=val_test_transforms)
    # print('Dataset', train_dataset[0])
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print("Dataloaders created.")
    print(len(train_loader), len(val_loader), len(test_loader))
    return train_loader, val_loader, test_loader

def calculate_samples_per_class(class_distribution):
    min_count = min(class_distribution.values())
    samples_per_class = {label_to_int[cls]: min_count // count for cls, count in class_distribution.items()}
    return samples_per_class



def load_data(folder1):
    images, labels = load_images(folder1)
    train_set, val_set, test_set = split_dataset(images, labels)

    # Print class distribution
    train_class_distribution = count_class_distribution(train_set)
    val_class_distribution = count_class_distribution(val_set)
    test_class_distribution = count_class_distribution(test_set)

    print("Training set class distribution:", train_class_distribution)
    print("Validation set class distribution:", val_class_distribution)
    print("Test set class distribution:", test_class_distribution)
    sampling_factors = {'Matig': 1, 'Redelijk': 1, 'Slecht': 1, 'Dood': 1, 'Goed': 1, 'Zeer Slecht': 1}
    

#     sampling_factors = calculate_sampling_factors(train_set)
    print('Sampling factors: ', sampling_factors)
    train_loader, val_loader, test_loader = create_dataloaders(train_set, val_set, test_set, sampling_factors, batch_size=8)

    return train_loader, val_loader, test_loader

In [ ]:
def extract_patches(image, patch_size, stride):
    patches = []
    c, height, width = image.size()

    for y in range(0, height - patch_size[1] + 1, stride):
        for x in range(0, width - patch_size[0] + 1, stride):
            patch = image[:, y:y + patch_size[1], x:x + patch_size[0]]
            patches.append(patch)

    return patches

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_preds = []

    progress_bar = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in progress_bar:

        images = images.to(device)
        labels = labels.to(device)


        # Forward pass
        outputs = model(images)

        #For ViT
#         logits = outputs.logits  # Extract the logits
#         loss = criterion(logits, labels)

        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        # _, predicted = torch.max(outputs, 1)
        _, predicted = torch.max(outputs, 1)

        correct = (predicted == labels).sum().item()
        progress_bar.set_postfix(loss=loss.item(), accuracy=correct/len(labels))

        total_loss += loss.item()
        total_correct += correct
        total_samples += labels.size(0)

        # For F1 score calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples
    f1 = f1_score(all_labels, all_preds, average='weighted')
    per_class_f1 = f1_score(all_labels, all_preds, average=None)

    return avg_loss, accuracy, f1, per_class_f1



def validate_or_test(model, loader, criterion, device, patch_size, stride, desc='Val'):
    model.eval()
    total_loss = 0
    total_samples = 0
    all_labels = []
    all_preds = []

    progress_bar = tqdm(loader, desc=desc, leave=False)
    with torch.no_grad():
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            batch_preds = []

            for image in images:
                # Apply sliding window approach
                patches = extract_patches(image, patch_size, stride)
                patches = torch.stack(patches).to(device)

                # Aggregate predictions for each patch
                patch_outputs = model(patches)

                # Calculate mode for each patch prediction
                modes, _ = torch.mode(patch_outputs, dim=0)
                batch_preds.append(modes)

            batch_preds = torch.stack(batch_preds)
            loss = criterion(batch_preds, labels)
            total_loss += loss.item()

            _, predicted = torch.max(batch_preds, 1)
            total_samples += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    per_class_f1 = f1_score(all_labels, all_preds, average=None)

    conf_matrix = confusion_matrix(all_labels, all_preds)
    conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix_percentage, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()
    return avg_loss, accuracy, f1, per_class_f1

def train_and_evaluate(model, train_loader, val_loader, test_loader, model_name, num_epochs, patch_size=(224, 224), stride=30):
    # Criterion, Optimizer, and Scheduler
    weight_decay=0.01 #L2 Optimisation
    lr=2e-4
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    global best_val_f1
    global best_model_weights

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        train_loss, train_accuracy, train_f1, train_f1_per_class = train_one_epoch(model, train_loader, criterion, optimizer, device)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1 Score: {train_f1:.4f}')
        print(f'Train F1 Score Per Class ', train_f1_per_class)

        val_loss, val_accuracy, val_f1, val_f1_per_class = validate_or_test(model, val_loader, criterion, device, patch_size, stride, desc='Val')
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}')
        print(f'Val F1 Score Per Class ', val_f1_per_class)
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_weights = model.state_dict()  # Save the best model weights

        # Save intermediate model weights
        torch.save(model.state_dict(), '/kaggle/working/model_state.pth')

        scheduler.step()


    test_loss, test_accuracy, test_f1, test_f1_per_class = validate_or_test(model, test_loader, criterion, device, patch_size, stride, desc='Test')
    print(f'Test Loss: {test_loss:.3f}, Accuracy: {test_accuracy:.3f}, F1 Score: {test_f1:.3f}, F1 Score Per Class [{test_f1_per_class[0]:.3f}')
    print(f'Test F1 Score Per Class ', test_f1_per_class)

    # After training is complete, load the best model weights
    model.load_state_dict(best_model_weights)

    # Save the best model weights
    torch.save(model.state_dict(), '/kaggle/working/best_model_state.pth')
    return model

In [ ]:
# Call the main function with the path to folder1 (after appending augmentated datasets to orginal dataset --final dataset called blurred-trees_withdataaugmentation)
imagesRoot = '/kaggle/input/blurred-trees-withdataaugmentation/tree'

path_to_images = imagesRoot 
train_loader, val_loader, test_loader = load_data(path_to_images)

# def load_pretrained_vit(num_labels):
#  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in2
class CNNWithFC(nn.Module):
    def __init__(self, num_classes):
        super(CNNWithFC, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
#         self.conv3= nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1)
        
        # Calculate input size for fully connected layers
        self._calculate_conv_output_size()
        
        # Define fully connected layers
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Apply convolutional layers
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
#         x = F.relu(self.conv3(x))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)
        # Flatten the feature map
        x = x.view(x.size(0), -1)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _calculate_conv_output_size(self):
        # Forward pass to calculate the output size after convolutional layers
        with torch.no_grad():
            x = torch.zeros(1, 3, 224, 224)  # Assuming input image size is 224x224
            x = self.conv1(x)
            x = F.max_pool2d(x, kernel_size=2, stride=2)
            x = self.conv2(x)
            x = F.max_pool2d(x, kernel_size=2, stride=2)
#             x = self.conv3(x)
#             x = F.max_pool2d(x, kernel_size=2, stride=2)
            self.fc_input_size = x.size(1) * x.size(2) * x.size(3)
def load_pretrained_cnn_with_fc(num_labels):
    # Define the CNN architecture
        # Instantiate the CNN model with fully connected layers
    model = CNNWithFC(num_labels)
    return model
num_classes = 6  
num_epochs = 5
# Load the pretrained ResNet-50 model
# model = models.resnet50(pretrained=True)
# vit_model = load_pretrained_vit(num_labels=num_classes)
cnn_model = CNNWithFC(num_classes=num_classes)

# model.fc = nn.Linear(model.fc.in_features, num_classes)
# Initialize best F1 score for validation

best_val_f1 = 0.0
best_model_weights = None
model_name = 'CNNwithFC'
trained_model = train_and_evaluate(cnn_model, train_loader, val_loader, test_loader, model_name, num_epochs)